In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Import data
df = pd.read_csv("medical_examination.csv")

# Add 'overweight' column
df['overweight'] = np.where((df["weight"] / ((df["height"] / 100) ** 2)) > 25, 1, 0)

# Normalize data by making 0 always good and 1 always bad. If the value of 'cholesterol' or 'gluc' is 1, make the value 0. If the value is more than 1, make the value 1.

for column in ["cholesterol", "gluc"]:
    df[column] = np.where(df[column] > 1, 1, 0)

# Draw Categorical Plot
def draw_cat_plot():
    # Create DataFrame for cat plot using `pd.melt` using just the values from 'cholesterol', 'gluc', 'smoke', 'alco', 'active', and 'overweight'.
    df_cat = pd.melt(df, id_vars=["id", "age", "gender", "height", "weight","ap_hi", "ap_lo", "cardio"])
    df_cat["cardio"] = df_cat["cardio"].astype("category")
    df_cat["value"] = df_cat["value"].astype("category")


    # Group and reformat the data to split it by 'cardio'. Show the counts of each feature. You will have to rename one of the columns for the catplot to work correctly.
    df_cat = df_cat.groupby(["cardio", "variable", "value"])[["value"]].count()
    df_cat = df_cat.rename(columns={"value":"total"}).reset_index()

    # Draw the catplot with 'sns.catplot()'
    
    graph = sns.catplot(data = df_cat, x = "variable", y = "total", hue = "value", kind="bar", col="cardio")
    fig = graph.fig

    # Do not modify the next two lines
    fig.savefig('catplot.png')
    return fig


# Draw Heat Map
def draw_heat_map():
    # Clean the data
    df_heat = df[~(df["ap_lo"] > df["ap_hi"])]
    df_heat = df_heat[~(df_heat["height"] < df["height"].quantile(q=0.025))]
    df_heat = df_heat[~(df_heat["height"] > df["height"].quantile(q=0.975))]
    df_heat = df_heat[~(df_heat["weight"] < df["weight"].quantile(q=0.025))]
    df_heat = df_heat[~(df_heat["weight"] > df["weight"].quantile(q=0.975))]

    # Calculate the correlation matrix
    corr = df_heat.corr().round(decimals=1)

    # Generate a mask for the upper triangle
    mask = np.triu(np.ones_like(corr, dtype = bool))



    # Set up the matplotlib figure
    fig, ax = plt.subplots(figsize = (11, 9))

    # Draw the heatmap with 'sns.heatmap()'

    sns.heatmap(data = corr, mask=mask, annot=True, square=True, fmt = ".1f")

    # Do not modify the next two lines
    fig.savefig('heatmap.png')
    return fig
